# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-12-27 05:38:20] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-27 05:38:20] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-27 05:38:20] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-27 05:38:22] WARNING server_args.py:1543: Attention backend not specified. Use fa3 backend by default.


[2025-12-27 05:38:22] INFO server_args.py:2420: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=66.93 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=59.00 GB):  20%|██        | 4/20 [00:00<00:01, 14.34it/s]

Capturing batches (bs=40 avail_mem=58.97 GB):  50%|█████     | 10/20 [00:00<00:00, 21.38it/s]

Capturing batches (bs=8 avail_mem=58.95 GB):  80%|████████  | 16/20 [00:00<00:00, 20.80it/s] 

Capturing batches (bs=1 avail_mem=58.93 GB): 100%|██████████| 20/20 [00:00<00:00, 20.60it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Ayla and I’m a High School Student from the Philippines. I am currently a Senior at the University of the Philippines in Barangay Pilos, Manila, Philippines. I attend the Department of Science and Technology, Engineering and Technology (DSTET) where I am a member of the Departmental Technical Committee (DTC), which is responsible for the promotion and facilitation of technology in the country. My passion is to be a technology leader and advocate. As a member of the DTC, I am responsible for identifying needs and needs for the public and government agencies and working to make technology accessible to those who need it. My goal
Prompt: The president of the United States is
Generated text:  a powerful figure that affects the lives of Americans. The American political system is a model for other countries. The president is elected to a term of four years. Most people assume that the president is the leader of the United States. The president does

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [Age] year old [Occupation]. I am a [Skill] who has always been [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and I am [Positive Trait]. I am [Positive Trait] and

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, also known as the City of Light. It is a historic city with a rich cultural heritage and is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also known for its vibrant nightlife, fashion industry, and world-renowned cuisine. The city is a major transportation hub and is a popular tourist destination, attracting millions of visit

Generated text:  likely to be characterized by several key trends:

1. Increased automation: AI is expected to become more and more integrated into various industries, leading to increased automation of tasks. This could result in the creation of new jobs, but also create new opportunities for people to work in areas like data analysis, machine learning, and software development.

2. Improved privacy and security: As AI systems become more sophisticated, there will be an increased need for measures to protect user data and prevent misuse. This could lead to the development of new privacy and security standards, as well as the adoption of more advanced encryption techniques.

3. Enhanced human-computer interaction:



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert fictional character's name], and I'm a [insert fictional character's profession or role]. I enjoy [insert one or two adjectives describing my personality or activities]. I'm always [insert one or two adjectives describing my appearance or personality]. I love [insert one or two adjectives describing my hobbies or interests]. I have a love for [insert one or two adjectives describing my hobbies or interests]. I'm a [insert fictional character's age], [insert fictional character's height], [insert fictional character's weight]. I have an impressive collection of [insert one or two items or objects related to hobbies or interests].

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the most populous city and the largest city in Europe, and the seat of the French government. It is also the birthplace 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 name

],

...

:

 Chat

bots

 are

 expected

In [6]:
llm.shutdown()